In [ ]:
from IPython.display import display
import serialhub
display(serialhub.__version__)
W = serialhub.SerialHubWidget()
display(W)
W.serial_options = { 'baudRate': 115200 }

In [ ]:
display(W.is_supported)
W.value = "CLEARED"
s = f"Test {W.status}"
W.write_str(s)
s

In [ ]:
W.pkt_send_front

In [ ]:
W.pkt_recv_front

In [ ]:
SER = serialhub.SerialIO(W)
SER.closed()

In [ ]:
CNT=0
def gotbuf(buf):
    global CNT
    CNT = CNT + 1
    W.value = f"Value now {CNT}"

W.on_recv(gotbuf)

In [ ]:
CNT

In [ ]:
W.on_recv(None)